In [57]:
import pandas as pd
import geopy.distance

In [61]:
listings = pd.read_csv("data/listings.csv")

listings_cleaned = listings[["id", "name", "neighbourhood_cleansed", "latitude", "longitude", "property_type",
                             "room_type", "accommodates", "bedrooms", "price","number_of_reviews",
                             "review_scores_rating", "review_scores_location", "reviews_per_month"]]
listings_cleaned.head()

poi = pd.read_csv("data/SEHENSWUERDIGOGD.csv")
poi['SHAPE'] = poi['SHAPE'].apply(lambda x: str(x).split('(')[1]
                                                          .split(')')[0]
                                                         .split(' '))
poi['longitude'] = poi['SHAPE'].apply(lambda x: x[0])
poi['latitude'] = poi['SHAPE'].apply(lambda x: x[1])
poi_cleaned = poi[['OBJECTID', 'NAME', 'latitude', 'longitude']].copy()
poi_cleaned.rename(columns={'OBJECTID':'objectid', 'NAME':'name'}, inplace=True)
poi_cleaned.head()

,objectid,name,latitude,longitude
0,436059,Sigmund Freud Museum,48.21869354516781,16.36299908389642
1,436060,Technisches Museum,48.1909187099549,16.31790543758023
2,436061,Wien Museum Karlsplatz,48.19925219338192,16.372965687537953
3,436062,Wagner:Werk Museum Postsparkasse,48.20997976936334,16.3811108370231
4,436063,Palais Epstein,48.20656531674478,16.359629184319658


In [62]:
listings_cleaned.head()

,id,name,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month
0,15883,b&b near Old Danube river,Donaustadt,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,1.0,$110.00,14,4.71,4.71,0.15
1,38768,central cityapartement- wifi- nice neighbourhood,Leopoldstadt,48.21924,16.37831,Entire rental unit,Entire home/apt,5,1.0,$69.00,350,4.75,4.75,2.50
2,40625,"Near Palace Schönbrunn, Apt. 1",Rudolfsheim-Fnfhaus,48.18434,16.32701,Entire rental unit,Entire home/apt,6,2.0,$145.00,181,4.83,4.59,1.23
3,392757,VCA3 Palais Brambilla - studio with city views,Innere Stadt,48.21496,16.37161,Entire rental unit,Entire home/apt,2,1.0,$100.00,100,4.64,4.89,0.79
4,51287,little studio- next to citycenter- wifi- nice ...,Leopoldstadt,48.21778,16.37847,Entire rental unit,Entire home/apt,3,NaN,$68.00,347,4.65,4.86,2.45


In [83]:
joined_df = poi_cleaned.join(listings_cleaned, how="cross", lsuffix="_poi", rsuffix="_listing")

joined_df['distance'] = joined_df.apply(lambda x: geopy.distance.geodesic((x['longitude_poi'], x['latitude_poi']),
                                                (x['longitude_listing'], x['latitude_listing'])).km, axis=1)
joined_df.head()

,objectid,name_poi,latitude_poi,longitude_poi,id,name_listing,neighbourhood_cleansed,latitude_listing,longitude_listing,property_type,room_type,accommodates,bedrooms,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month,distance
0,436059,Sigmund Freud Museum,48.21869354516781,16.36299908389642,15883,b&b near Old Danube river,Donaustadt,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,1.0,$110.00,14,4.71,4.71,0.15,7.599359
1,436059,Sigmund Freud Museum,48.21869354516781,16.36299908389642,38768,central cityapartement- wifi- nice neighbourhood,Leopoldstadt,48.21924,16.37831,Entire rental unit,Entire home/apt,5,1.0,$69.00,350,4.75,4.75,2.50,1.695350
2,436059,Sigmund Freud Museum,48.21869354516781,16.36299908389642,40625,"Near Palace Schönbrunn, Apt. 1",Rudolfsheim-Fnfhaus,48.18434,16.32701,Entire rental unit,Entire home/apt,6,2.0,$145.00,181,4.83,4.59,1.23,5.416173
3,436059,Sigmund Freud Museum,48.21869354516781,16.36299908389642,392757,VCA3 Palais Brambilla - studio with city views,Innere Stadt,48.21496,16.37161,Entire rental unit,Entire home/apt,2,1.0,$100.00,100,4.64,4.89,0.79,1.033022
4,436059,Sigmund Freud Museum,48.21869354516781,16.36299908389642,51287,little studio- next to citycenter- wifi- nice ...,Leopoldstadt,48.21778,16.37847,Entire rental unit,Entire home/apt,3,NaN,$68.00,347,4.65,4.86,2.45,1.714831
